# Create High-k Metal Gate Stack Tutorial

This notebook demonstrates how to create a high-k metal gate stack heterostructure with four materials: Si (substrate), SiO2 (gate oxide), HfO2 (high-k dielectric), and TiN (metal gate).

We'll create a representation of the material from the [QuantumATK tutorial](https://docs.quantumatk.com/tutorials/hkmg_builder/hkmg_builder.html) without the amorphous step.

<img src="https://docs.quantumatk.com/_images/update_square_default_stack.png" alt="High-k Metal Gate Stack" width="700"/>


## 1. Configuration Parameters


In [ ]:
# Global parameters
MAX_AREA = 200  # Maximum area for strain matching
MAX_AREA_RATIO_TOL = 0.25  # Maximum area ratio tolerance for strain matching
MAX_ANGLE_TOLERANCE = 0.15  # Maximum angle tolerance for strain matching
MAX_LENGTH_TOLERANCE = 0.15  # Maximum length tolerance for strain matching

# Structure parameters for each layer
STRUCTURE_PARAMS = [
    {
        # Silicon substrate
        "slab_params": {
            "miller_indices": (1, 0, 0),
            "thickness": 4,  # atomic layers
            "vacuum": 5.0,  # Angstroms
            "use_orthogonal_z": True
        },
        "interface_distance": None  # No interface for substrate
    },
    {
        # SiO2 layer
        "slab_params": {
            "miller_indices": (1, 0, 0),
            "thickness": 3,
            "vacuum": 5.0,
            "use_orthogonal_z": True
        },
        "interface_distance": 2.5,  # Distance to Si substrate
        "vacuum": 0.5  # Vacuum for SiO2-terminating interface layer
    },
    {
        # HfO2 layer
        "slab_params": {
            "miller_indices": (0, 0, 1),
            "thickness": 2,
            "vacuum": 0.5,
            "xy_supercell_matrix": [[2, 0], [0, 2]],
            "use_orthogonal_z": True
        },
        "interface_distance": 3.0,  # Distance to SiO2
        "vacuum": 0.5  # Vacuum for HfO2-terminating interface layer
    },
    {
        # TiN layer
        "slab_params": {
            "miller_indices": (0, 0, 1),
            "thickness": 3,
            "vacuum": 5.0,  # Add vacuum to final layer
            "use_orthogonal_z": True,
            "xy_supercell_matrix": [[3, 0], [0, 3]]
        },
        "interface_distance": 2.5,  # Distance to HfO2
        "vacuum": 10.0  # Vacuum for TiN-terminating interface layer
    }
]

INTERFACE_1_INDEX = 11 # was found in the previous runs

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")

### 1.3. Get input material
In this notebook we will use materials from Standata.

In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials

materials_1 = [
    Material.create(Materials.get_by_name_first_match("SiO2")),  # SiO2
    Material.create(Materials.get_by_name_first_match("Silicon")),  # Si substrate
]
materials_2 = [
    Material.create(Materials.get_by_name_first_match("HfO2.*MCL")),  # HfO2
    Material.create(Materials.get_by_name_first_match("TiN")),  # TiN
]

## 2. Create Si/SiO2 Interface
### 2.1. Create Si/SiO2 Interface

In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab, \
    PymatgenSlabGeneratorParameters
from mat3ra.made.tools.build.interface import (
    InterfaceConfiguration,
    ZSLStrainMatchingParameters,
    ZSLStrainMatchingInterfaceBuilder,
    ZSLStrainMatchingInterfaceBuilderParameters
)
from utils.visualize import visualize_materials as visualize

film_1_params = STRUCTURE_PARAMS[1]
substrate_1_params = STRUCTURE_PARAMS[0]

film_1_config = SlabConfiguration(
    bulk=materials_1[1],
    **film_1_params["slab_params"]
)

substrate_1_config = SlabConfiguration(
    bulk=materials_1[0],
    **substrate_1_params["slab_params"]
)
params = PymatgenSlabGeneratorParameters(symmetrize=False)

film_1_terminations = get_terminations(film_1_config, build_parameters=params)
substrate_terminations = get_terminations(substrate_1_config, build_parameters=params)

film_termination = film_1_terminations[0]
substrate_termination = substrate_terminations[0]

interface_config = InterfaceConfiguration(
    film_configuration=film_1_config,
    substrate_configuration=substrate_1_config,
    film_termination=film_termination,
    substrate_termination=substrate_termination,
    distance_z=film_1_params["interface_distance"],
    vacuum=film_1_params["vacuum"],
)

builder = ZSLStrainMatchingInterfaceBuilder(
    build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
        strain_matching_parameters=ZSLStrainMatchingParameters(max_area=MAX_AREA, max_area_ratio_tol=MAX_AREA_RATIO_TOL,
                                                               max_angle_tol=MAX_ANGLE_TOLERANCE,
                                                               max_length_tol=MAX_LENGTH_TOLERANCE)
    )
)

interfaces = builder.get_materials(configuration=interface_config)
interface_1 = interfaces[INTERFACE_1_INDEX]

### 2.2. Visualize the Si/SiO2 Interface

In [ ]:
visualize(
    interface_1,
    repetitions=[1, 1, 1],
    title="Si/SiO2 Interface"
)

visualize(
    interface_1,
    repetitions=[1, 1, 1],
    title="Si/SiO2 Interface",
    rotation='-90x'
)


## 3. Add HfO2 Layer
### 3.1. Add a layer with SimpleInterfaceBuilder

In [ ]:
from mat3ra.made.tools.build.interface import SimpleInterfaceBuilder, SimpleInterfaceBuilderParameters

film_2_params = STRUCTURE_PARAMS[2]

film_2_config = SlabConfiguration(
    bulk=materials_2[0],
    **film_2_params["slab_params"]
)

hfo2_slab = create_slab(film_2_config, build_parameters=params)

film_2_config = SlabConfiguration(
    bulk=hfo2_slab,
    **film_2_params["slab_params"]
)

substrate_2_config = SlabConfiguration(
    bulk=interface_1,
    miller_indices=(0, 0, 1)
)

film_2_terminations = get_terminations(film_2_config, build_parameters=params)
substrate_2_terminations = get_terminations(substrate_2_config, build_parameters=params)

print("Film terminations:", film_2_terminations)
print("Substrate terminations:", substrate_2_terminations)

interface_2_config = InterfaceConfiguration(
    film_configuration=film_2_config,
    substrate_configuration=substrate_2_config,
    film_termination=film_2_terminations[0],
    substrate_termination=substrate_2_terminations[0],
    distance_z=film_2_params["interface_distance"],
    vacuum=film_2_params["vacuum"]
)

builder = SimpleInterfaceBuilder(build_parameters=SimpleInterfaceBuilderParameters(scale_film=True, build_slabs=False))
interface_2 = builder.get_material(configuration=interface_2_config)

### 3.2. Visualize the Si/SiO2/HfO2 Interface

In [ ]:
visualize(
    interface_2,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2 Interface"
)

visualize(
    interface_2,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2 Interface",
    rotation='-90x'
)

## 4. Add TiN Layer
### 4.1. Create a TiN Slab

In [ ]:
from mat3ra.made.tools.build.supercell import create_supercell

film_3_params = STRUCTURE_PARAMS[3]

interface_2_flipped = create_supercell(interface_2, scaling_factor=[1, -1, -1])

tin_film_config = SlabConfiguration(
    bulk=materials_2[1],
    **film_3_params["slab_params"]
)

tin_slab = create_slab(tin_film_config, build_parameters=params)

### 4.2. Get terminations for TiN and Si/SiO2/HfO2

In [ ]:
film_3_config = SlabConfiguration(
    bulk=tin_slab,
    use_orthogonal_z=True,
    use_conventional_cell=False,
)

substrate_3_config = SlabConfiguration(
    bulk=interface_2_flipped,
    miller_indices=(0, 0, 1)
)

film_3_terminations = get_terminations(film_3_config, build_parameters=params)
substrate_3_terminations = get_terminations(substrate_3_config, build_parameters=params)

print("Film terminations:", film_3_terminations)
print("Substrate terminations:", substrate_3_terminations)

### 4.3. Add TiN layer with SimpleInterfaceBuilder

In [ ]:
interface_3_config = InterfaceConfiguration(
    film_configuration=film_3_config,
    substrate_configuration=substrate_3_config,
    film_termination=film_3_terminations[0],
    substrate_termination=substrate_3_terminations[0],
    distance_z=film_3_params["interface_distance"],
    vacuum=film_3_params["vacuum"]
)


builder = SimpleInterfaceBuilder(build_parameters=SimpleInterfaceBuilderParameters(scale_film=True, create_slabs=False))

interface_3 = builder.get_material(configuration=interface_3_config)
# invert the interface along the z-axis to match example
interface_3 = create_supercell(interface_3, scaling_factor=[1, -1, -1])

### 4.2. Visualize the Si/SiO2/HfO2/TiN Interface

In [ ]:
visualize(
    interface_3,
    title="Si/SiO2/HfO2/TiN Interface",
    viewer="wave",
    rotation='-90x'
)

## 5. Save final material


In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials
interface_3.name = "Si-SiO2-HfO2-TiN Heterostructure"
set_materials(interface_3)
download_content_to_file(interface_3, "heterostructure_high_k_metal_gate.json")